# Text summarizer
This is my playground to mess around with. Do not take this code seriously 

### All imports

In [1]:
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
)
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
)

/home/jovyan/Text-summarizer/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jovyan/Text-summarizer/.conda/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


### Preparing Dataset

In [2]:
# Check if GPU is available
device = 0 if torch.cuda.is_available() else -1

# Load the dataset
file_path = "../Dataset/wikihowSep.csv"
df = pd.read_csv(file_path)

# Ensure the dataset has the correct columns
if "text" not in df.columns or "headline" not in df.columns:
    raise ValueError("The dataset must contain 'text' and 'headline' columns")

# Transform the dataset into a Hugging Face dataset
dataset = Dataset.from_pandas(df[["text", "headline"]])

# Display the first two rows to verify the data
print(df.head(2))

# Print out row 1
print(df.iloc[1])

# Load the tokenizer and model
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


# Define the preprocess function
def preprocess_function(examples):
    inputs = examples["text"]
    inputs = [str(i) for i in inputs]  # Ensure inputs are strings
    model_inputs = tokenizer(
        inputs, max_length=512, truncation=True, padding="max_length"
    )

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            [str(i) for i in examples["headline"]],
            max_length=150,
            truncation=True,
            padding="max_length",
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Apply the preprocess function to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Use a DataCollator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

                                            overview  \
0   So you're a new or aspiring artist and your c...   
1   If you want to be well-read, then, in the wor...   

                           headline  \
0            \nSell yourself first.   
1  \nRead the classics before 1600.   

                                                text          sectionLabel  \
0   Before doing anything else, stop and sum up y...                 Steps   
1   Reading the classics is the very first thing ...  Reading the Classics   

                         title  
0  How to Sell Fine Art Online  
1          How to Be Well Read  
overview         If you want to be well-read, then, in the wor...
headline                         \nRead the classics before 1600.
text             Reading the classics is the very first thing ...
sectionLabel                                 Reading the Classics
title                                         How to Be Well Read
Name: 1, dtype: object


/home/jovyan/Text-summarizer/.conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map:   0%|          | 0/1585695 [00:00<?, ? examples/s]/home/jovyan/Text-summarizer/.conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1585695/1585695 [03:35<00:00, 7355.82 examples/s]


In [3]:
training_args = Seq2SeqTrainingArguments(
    output_dir="base",
    evaluation_strategy="steps",
    eval_steps=20,  # Evaluate every 20 steps
    logging_strategy="steps",
    logging_steps=20,  # Log every 20 steps
    save_strategy="steps",
    save_steps=100,  # Save the model every 100 steps
    per_device_train_batch_size=8,
    gradient_accumulation_steps=8,
    save_total_limit=5,
    max_steps=100,  # Run training for 100 steps
    weight_decay=0.01,
    gradient_checkpointing=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    predict_with_generate=True,
    optim="adafactor",
    bf16=True,
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/home/jovyan/Text-summarizer/.conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [4]:
# Fine-tune the model
trainer.train()

/home/jovyan/Text-summarizer/.conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


KeyboardInterrupt: 